In [72]:
import pandas as pd
from datetime import datetime
import os
import numpy as np

energy = pd.read_excel(
    'data/Energy Indicators.xls', 
    skiprows=17, 
    skipfooter=1, 
    usecols='C:F'  
)

In [73]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 2  247 non-null    object 
 1   Petajoules  227 non-null    object 
 2   Gigajoules  227 non-null    object 
 3   %           227 non-null    float64
dtypes: float64(1), object(3)
memory usage: 8.4+ KB


In [74]:
energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']


In [75]:
energy['Energy Supply'] = energy['Energy Supply'] * 1_000_000


In [76]:
def replace_dots_with_nan(value):
    if isinstance(value, str):
        if re.fullmatch(r'\.*', value):
            return np.NaN
    return value
energy['Energy Supply'] = energy['Energy Supply'].apply(replace_dots_with_nan).astype('float64')
energy['Energy Supply per Capita'] = energy['Energy Supply per Capita'].apply(replace_dots_with_nan).astype('float64')

In [77]:
country_new_names = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"
}

energy['Country'] = energy['Country'].replace(country_new_names)

In [78]:
import re

def clean_country_name(country):
    cleaned = re.sub(r'\(.*\)', '', country)
    cleaned = re.sub(r'\d+', '', cleaned)
    cleaned = cleaned.strip()
    return cleaned

energy['Country'] = energy['Country'].astype(str)

energy['Country'] = energy['Country'].apply(clean_country_name)

In [79]:
energy

,Country,Energy Supply,Energy Supply per Capita,% Renewable
0,Afghanistan,3.210000e+08,10.0,78.669280
1,Albania,1.020000e+08,35.0,100.000000
2,Algeria,1.959000e+09,51.0,0.551010
3,American Samoa,NaN,NaN,0.641026
4,Andorra,9.000000e+06,121.0,88.695650
...,...,...,...,...
259,nan,NaN,NaN,NaN
260,nan,NaN,NaN,NaN
261,nan,NaN,NaN,NaN
262,nan,NaN,NaN,NaN


In [80]:
gdp = pd.read_csv('data/world_bank.csv', skiprows=4)

gdp_country_rename = {
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}

gdp['Country Name'] = gdp['Country Name'].replace(gdp_country_rename)

In [81]:
scimen = pd.read_excel('data/scimagojr-3.xlsx')

In [82]:
df_energy_scimen_gdp = pd.merge(scimen, energy, left_on='Country', right_on='Country', how='inner')
df_energy_scimen_gdp = pd.merge(df_energy_scimen_gdp, gdp, left_on='Country', right_on='Country Name', how='inner')

df_energy_scimen_gdp.set_index('Country', inplace=True)

In [120]:
df_energy_scimen_gdp.columns

Index(['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       'Self-Citations to Total Citations Ratio', 'HighRenew'],
      dtype='object')

In [84]:
scimen_count = len(scimen)
gdp_count = len(gdp)
energy_count = len(energy)


In [85]:
energy_scimen_gdp_count = min(scimen_count, energy_count, gdp_count)

print("Initial Counts:", energy_scimen_gdp_count)

Initial Counts: 191


In [86]:
merged_data_count = len(df_energy_scimen_gdp)
entries_lost = energy_scimen_gdp_count - merged_data_count 


print("Number of entries lost:", entries_lost) 

Number of entries lost: 33


In [90]:
df_energy_scimen_gdp

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.75491,...,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.23282,...,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.28868,...,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.94543,...,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.90153,...,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Maldives,183,1,1,1,0,1.00,1,1.500000e+07,43.0,0.00000,...,1.862196e+09,2.063143e+09,2.320557e+09,2.192661e+09,2.323402e+09,2.525755e+09,2.588810e+09,2.710479e+09,2.886134e+09,2.929826e+09
Belize,185,1,1,6,0,6.00,1,1.300000e+07,39.0,64.69003,...,1.285312e+09,1.299522e+09,1.341485e+09,1.352206e+09,1.397113e+09,1.426484e+09,1.479846e+09,1.499280e+09,1.560479e+09,1.590417e+09
Palau,186,1,1,0,0,0.00,0,3.000000e+06,152.0,18.75000,...,2.074094e+08,2.072990e+08,1.957150e+08,1.779528e+08,1.838000e+08,1.930672e+08,1.992454e+08,1.943911e+08,2.026654e+08,2.216412e+08


In [92]:
required_columns = [
    'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
    'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita',
    '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'
]

df_energy_scimen_gdp = df_energy_scimen_gdp[required_columns]


In [93]:
df_energy_scimen_gdp.columns

Index(['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'],
      dtype='object')

In [102]:
top_15 = df_energy_scimen_gdp[df_energy_scimen_gdp['Rank'] <= 15]


In [103]:
gdp_columns = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

avgGDP = top_15[gdp_columns].mean(axis=1, skipna=True)

In [104]:
top_15

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93.0,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149.0,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214.0,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296.0,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165.0,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26.0,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166.0,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12
South Korea,10,11983,11923,114675,22595,9.57,104,1.100700e+10,221.0,2.279353,9.410199e+11,9.924316e+11,1.020510e+12,1.027730e+12,1.094499e+12,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12
Italy,11,10964,10794,111850,26661,10.20,106,6.530000e+09,109.0,33.667230,2.202170e+12,2.234627e+12,2.211154e+12,2.089938e+12,2.125185e+12,2.137439e+12,2.077184e+12,2.040871e+12,2.033868e+12,2.049316e+12


In [105]:
avgGDP.index = top_15.index

avgGDP = avgGDP.sort_values(ascending=False)

In [106]:
avgGDP

Country
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

In [107]:
mean_energy_supply_per_capita = df_energy_scimen_gdp['Energy Supply per Capita'].mean()

print("Mean Energy Supply per Capita:", mean_energy_supply_per_capita)


Mean Energy Supply per Capita: 99.92356687898089


In [108]:
country_with_max_renewable = df_energy_scimen_gdp['% Renewable'].idxmax()

max_renewable = df_energy_scimen_gdp['% Renewable'].max()

result = (country_with_max_renewable, max_renewable)

In [109]:
result

('Albania', 100.0)

In [110]:
df_energy_scimen_gdp['Self-Citations to Total Citations Ratio'] = df_energy_scimen_gdp['Self-citations'] / df_energy_scimen_gdp['Citations']


/tmp/ipykernel_7049/3276849165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_energy_scimen_gdp['Self-Citations to Total Citations Ratio'] = df_energy_scimen_gdp['Self-citations'] / df_energy_scimen_gdp['Citations']


In [111]:
max_ratio = df_energy_scimen_gdp['Self-Citations to Total Citations Ratio'].max()

country_with_max_ratio = df_energy_scimen_gdp['Self-Citations to Total Citations Ratio'].idxmax()
result = (country_with_max_ratio, max_ratio)

In [112]:
result

('China', 0.6893126179389422)

In [113]:
renewable_median = df_energy_scimen_gdp['% Renewable'].median()


In [116]:
df_energy_scimen_gdp['HighRenew'] = (df_energy_scimen_gdp['% Renewable'] >= renewable_median).astype(int)


/tmp/ipykernel_7049/3591405839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_energy_scimen_gdp['HighRenew'] = (df_energy_scimen_gdp['% Renewable'] >= renewable_median).astype(int)


In [117]:
high_renew_series = df_energy_scimen_gdp['HighRenew']
high_renew_series = high_renew_series.sort_index()

In [118]:
high_renew_series

Country
Afghanistan             1
Albania                 1
Algeria                 0
American Samoa          0
Andorra                 1
                       ..
Ukraine                 0
United Arab Emirates    0
Uruguay                 1
Uzbekistan              1
Zimbabwe                1
Name: HighRenew, Length: 158, dtype: int64

In [124]:
ContinentDict = {
    'China': 'Asia',
    'United States': 'North America',
    'Japan': 'Asia',
    'United Kingdom': 'Europe',
    'Russian Federation': 'Europe',
    'Canada': 'North America',
    'Germany': 'Europe',
    'India': 'Asia',
    'France': 'Europe',
    'South Korea': 'Asia',
    'Italy': 'Europe',
    'Spain': 'Europe',
    'Iran': 'Asia',
    'Australia': 'Australia',
    'Brazil': 'South America'
}

df_energy_scimen_gdp['Continent'] = df_energy_scimen_gdp.index.map(ContinentDict)


/tmp/ipykernel_7049/1177601460.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_energy_scimen_gdp['Continent'] = df_energy_scimen_gdp.index.map(ContinentDict)


In [125]:
df_energy_scimen_gdp['Estimated Population'] = df_energy_scimen_gdp['Energy Supply'] / df_energy_scimen_gdp['Energy Supply per Capita']


/tmp/ipykernel_7049/2631997650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_energy_scimen_gdp['Estimated Population'] = df_energy_scimen_gdp['Energy Supply'] / df_energy_scimen_gdp['Energy Supply per Capita']


In [126]:
continent_stats = df_energy_scimen_gdp.groupby('Continent')['Estimated Population'].agg(['size', 'sum', 'mean', 'std'])


In [127]:
continent_stats

,size,sum,mean,std
Continent,,,,
Asia,5,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1,2.331602e+07,2.331602e+07,NaN
Europe,5,3.940587e+08,7.881174e+07,3.813228e+07
North America,1,3.523986e+07,3.523986e+07,NaN
South America,1,2.059153e+08,2.059153e+08,NaN
